https://github.com/CompVis/latent-diffusion/blob/main/ldm/modules/diffusionmodules/model.py
위 링크의 model.py 파일 중, 아래의 부분을 필사 및 주석을 달아서 ipynb 파일로 제출해주시면 됩니다.

1. ResnetBlock.forward (121 ~ 141줄)
2. Model.forward (316 ~ 356줄)


In [ ]:
# Residual Block

def forward(self, x, temb):
    # residual connection을 위해 입력을 그대로 보존
    h = x

    # 1번째 정규화 (보통 GroupNorm)
    h = self.norm1(h)

    # 비선형 함수 (ex. SiLU / Swish)
    h = nonlinearity(h)

    # 1번째 convolution
    h = self.conv1(h)

    # timestep embedding이 있을 경우 feature map에 더해줌
    # temb: (B, C) → (B, C, 1, 1) 로 변환하여 spatial broadcast
    if temb is not None:
        h = h + self.temb_proj(nonlinearity(temb))[:, :, None, None]

    # 2번째 정규화
    h = self.norm2(h)

    # 비선형 함수
    h = nonlinearity(h)

    # dropout (과적합 방지)
    h = self.dropout(h)

    # 2번째 convolution
    h = self.conv2(h)

    # 입력 채널 수와 출력 채널 수가 다르면 shortcut을 맞춰줌
    if self.in_channels != self.out_channels:
        if self.use_conv_shortcut:
            # conv 기반 shortcut
            x = self.conv_shortcut(x)
        else:
            # 1x1 convolution (Network-in-Network)
            x = self.nin_shortcut(x)

    # residual connection: 입력 x + 변환된 h
    return x + h


In [ ]:
# Diffusion U-Net 전체 forward

def forward(self, x, t=None, context=None):
    # context가 있으면 channel 방향으로 concat (conditional diffusion)
    if context is not None:
        x = torch.cat((x, context), dim=1)

    # timestep embedding 사용 여부
    if self.use_timestep:
        # timestep conditioning을 쓰는 경우 t는 반드시 필요
        assert t is not None

        # timestep t → sinusoidal embedding
        temb = get_timestep_embedding(t, self.ch)

        # timestep embedding을 MLP에 통과
        temb = self.temb.dense[0](temb)
        temb = nonlinearity(temb)
        temb = self.temb.dense[1](temb)
    else:
        temb = None

    # Downsampling path
    # 입력을 초기 convolution에 통과
    hs = [self.conv_in(x)]

    for i_level in range(self.num_resolutions):
        for i_block in range(self.num_res_blocks):
            # ResBlock (timestep embedding 조건부)
            h = self.down[i_level].block[i_block](hs[-1], temb)

            # Attention block이 있으면 적용
            if len(self.down[i_level].attn) > 0:
                h = self.down[i_level].attn[i_block](h)

            # skip connection을 위해 중간 feature 저장
            hs.append(h)

        # 마지막 resolution이 아니면 downsample 수행
        if i_level != self.num_resolutions - 1:
            hs.append(self.down[i_level].downsample(hs[-1]))


    # Middle block
    # 가장 낮은 해상도의 feature
    h = hs[-1]

    # ResBlock → Attention → ResBlock
    h = self.mid.block_1(h, temb)
    h = self.mid.attn_1(h)
    h = self.mid.block_2(h, temb)

    # Upsampling path
    for i_level in reversed(range(self.num_resolutions)):
        for i_block in range(self.num_res_blocks + 1):
            # Downsampling 단계에서 저장한 feature와 concat (skip connection)
            h = self.up[i_level].block[i_block](
                torch.cat([h, hs.pop()], dim=1), temb
            )

            # Attention block 적용 (있을 경우)
            if len(self.up[i_level].attn) > 0:
                h = self.up[i_level].attn[i_block](h)

        # 최상위 resolution이 아니면 upsample
        if i_level != 0:
            h = self.up[i_level].upsample(h)

    # 최종 출력 feature map 반환
    return h
